In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install transformers[torch]
!pip install datasets

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BlenderbotTokenizer
# from datasets import load_dataset

model_name = "./saved_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import pandas as pd

df = pd.read_json("./translate_data.json")

In [4]:
sequences = list(df["examples"])

In [5]:
# sequences = ['Who are you?',
#              'What is your name?',
#              'Are you ok?',
#              'I got flu',
#              "KSK01 prints without data NB",
#              'I got cough iam a winner. To go to hospital, turn left, turn right and then go straight ahead']

batch = tokenizer(sequences, padding='max_length', truncation=True, return_tensors='pt')
# batch = tokenizer(sequences, padding=True, return_tensors='pt')

batch

{'input_ids': tensor([[ 680,   58,   50,  ...,    0,    0,    0],
        [1786,  343, 2618,  ...,    0,    0,    0],
        [ 432,   41,  596,  ...,    0,    0,    0],
        ...,
        [ 553, 7826, 1099,  ...,    0,    0,    0],
        [ 553,  366, 3645,  ...,    0,    0,    0],
        [ 855,  983,  366,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [6]:
# tokenizer.decode(batch['input_ids'][3])

In [7]:
replies = list(df["response"])

In [8]:
replies

["Step 1: Please print criteria\nStep 2: The printing contract does not go to NB, then go to the list of the patient list that has been welcomed -> Find the company to see if the company has been welcomed KSK or not?\n'+ If not, do not report to the KSK -> need to report to the user\n+ If then, go to the medical record screen to check",
 '1/ The reception list screen: Check if NB has a service.If so, check the reception status of KSK = have welcomed KSK or not?-> If not, receive.If then set it to the case below\n2/ Has the account checked the specialties according to the value code in the general settings have the key chuyen_khoa?\n3/ Check if the examination service in the list of medical examination services has been assigned to KSK specialties',
 '1/ Ask for the get time that the user reports -> Check the implementation time (implementation time) of the service at the medical record screen, press the eye icon to see, see satisfied timeLis Get space yet?\n(1.1) If not, then report th

In [9]:
# replies = ["I am VIVI",
#            "my name is VIVI",
#            "I am good, how about you?",
#            "Should stay at home based on",
#            "Step 1: Please print criteria\nStep 2: The printing contract does not go to NB, then go to the list of the patient list that has been welcomed -> Find the company to see if the company has been welcomed KSK or not?\n'+ If not, do not report to the KSK -> need to report to the user\n+ If then, go to the medical record screen to check",
#            "Should go to hospital based on. To day is a beautiful day ever since before.\n I am the Strongest vietnamese"]

batch["labels"] = tokenizer(replies, padding='max_length', truncation=True, return_tensors='pt').input_ids

In [10]:
batch["labels"]

tensor([[5425,  497,   33,  ...,    0,    0,    0],
        [ 497,   22,  384,  ...,    0,    0,    0],
        [ 497,   22, 7472,  ...,  271, 4641,    2],
        ...,
        [2392,  281,  360,  ...,    0,    0,    0],
        [ 281,  372, 3645,  ...,    0,    0,    0],
        [6950,   19,  281,  ...,    0,    0,    0]])

In [11]:
from datasets import Dataset

ivi_dataset = Dataset.from_dict(batch)
ivi_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 83
})

In [12]:
special_token_dict = tokenizer.special_tokens_map
tokenizer.add_special_tokens(special_token_dict)

5

In [13]:
model.resize_token_embeddings(len(tokenizer))

Embedding(8009, 1280)

In [14]:
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [15]:
from transformers import TrainingArguments, DataCollatorWithPadding

training_args = TrainingArguments("test-trainer",
                                  num_train_epochs=100,
                                  # per_device_train_batch_size=4,
                                  # per_device_eval_batch_size=4,
                                  # gradient_accumulation_steps=4,

)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=ivi_dataset,
    eval_dataset=ivi_dataset,
    tokenizer=tokenizer
)

In [17]:
len("1/ Ask for the get time that the user reports -> Check the implementation time (implementation time) of the service at the medical record screen, press the eye icon to see, see satisfied timeLis Get space yet?\n(1.1) If not, then report the right time\n(1.2) If then check the case: Has the service category were set up the connection unit?If set up, then check the implementation room of the service is set out of the hospital = True or False -> True, the default does not display (need to change the settings and the repair impact of the implementation room to eat according to the designnew establishment), FASLE will display the side Lis\n+ If it is true (1.1), (1.2), then report the ISOFH check".split(" "))

123

In [18]:
trainer.train()

You're using a BlenderbotTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.451800
1000,0.001100


TrainOutput(global_step=1100, training_loss=0.20592531173743986, metrics={'train_runtime': 899.8005, 'train_samples_per_second': 9.224, 'train_steps_per_second': 1.222, 'total_flos': 2257969545216000.0, 'train_loss': 0.20592531173743986, 'epoch': 100.0})

In [19]:
saved_model = "./saved_model"

tokenizer.save_pretrained(saved_model)
model.save_pretrained(saved_model)


In [27]:
!pip install googletrans==4.0.0rc1

  Preparing metadata (setup.py) ... done
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=41659ed52cb66b06bd05b68d94f64101c9bc672349e7cfb5d9e28367a9cf89c9
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [3]:
from googletrans import Translator

def vi_to_en(text):
  translator = Translator()
  translation = translator.translate(text, src='vi', dest='en')
  return translation.text

def en_to_vi(text):
  translator = Translator()
  translation = translator.translate(text, src='en', dest='vi')
  return translation.text

In [4]:
vi_to_en("TÔi là sơn")

'I am Son'

In [5]:
input_text = vi_to_en("Bạn tên gì nhỉ")
tokenized_input = tokenizer(input_text, return_tensors='pt')
tokenized_input

{'input_ids': tensor([[ 714,  341,  414, 1356,   19, 1345,   79,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [9]:
device = "cpu"
inputs = tokenized_input['input_ids'].to(device)

In [10]:
tokenized_input = tokenized_input.to(device)

In [11]:
outputs = model.generate(**tokenized_input)
outputs

tensor([[   1,  863, 1356,  315,  799,  693,   80,   21,   48,  372,  404,  265,
          858, 6789,  287, 1998, 2498,  458,  414,  707, 3757, 4401,  319,  281,
         3323,   21,    2]])

In [12]:
en_to_vi(tokenizer.batch_decode(outputs)[0])

'<s> Tên tôi là Vivi.i chỉ là một bot được cài đặt để trả lời các câu hỏi về các tài liệu sử dụng của bạn mà tôi đã học. </s>'